# Attention -- implementing attention between a given decoder state and the encoder states

Below, there are two matrices, `encoder_states` and `decoder_states` representing the state of the hidden layer after processing each word by the encoder and the static embedding related to a given input of the decoder. A single hidden layer state contains an embedding of length = 3, which is equal to the size of the embedding in the decoder. In the encoder, we have 4 hidden layer states, because we are processing a sequence consisting of 4 tokens.

There are 5 tokens in the decoder, which are generated based on the sequence processed by the encoder.

The task is to: 

a) Calculate the similarity of all embeddings from the decoder (queries) to all embeddings of subsequent states of the encoder (keys) (remember that matrices can be transposed. In NumPy, we transpose a matrix using `matrix_name.T`)

b) Softmax (imported from scipy) should be performed on the created similarity matrix. Note: remember to apply softmax in the right dimension. All hidden states of the encoder should be softmaxed in the context of a given decoder state. In scipy, the softmax function includes an `axis` argument which may help.

c) Combine the attention matrix from step b) and `encoder_states` to generate a matrix containing context vectors for each token from the decoder.

---
**Done by:** Sofya Aksenyuk, 150284.

---

In [1]:
import numpy as np
from scipy.special import softmax

# scipy.special.softmax(x, axis=None)

encoder_states = np.array(
    [[1.2, 3.4, 5.6],   # encoder's hidden layer output at the step 1, related to a given input token, e.g., I
    [-2.3, 0.2, 7.2],   # encoder's hidden layer output at the step 2, related to a given token, e.g., like
    [10.2, 0.2, 0.3],   # encoder's hidden layer output at the step 3, related to a given token, e.g., NLP
    [0.4, 0.7, 1.2]]    # encoder's hidden layer output at the step 4, related to a given token, e.g., "."
)



decoder_states = np.array(
    [[0.74, 0.23, 0.56],  # decoder's static word embedding at the step 1, related to a given token, e.g., <BOS>
    [7.23, 0.12, 0.55],  # decoder's static word embedding at the step 2, related to a given token, e.g., Ja
    [9.12, 4.23, 0.44], # decoder's static word embedding at the step 3, related to a given token, e.g., lubię
    [4.1, 3.23, 0.5],    # decoder's static word embedding at the step 4, related to a given token, e.g., przetwarzanie
    [5.2, 3.1, 8.5]]     # decoder's static word embedding at the step 5, related to a given token, e.g., języka
)

In [2]:
from scipy import linalg, mat, dot

sim_mat= dot(decoder_states, encoder_states.T)
att_mat = softmax(sim_mat, axis=1)
cont_vec = dot(att_mat, encoder_states)

<ipython-input-2-6de68eb61f38>:3: DeprecationWarning: scipy.dot is deprecated and will be removed in SciPy 2.0.0, use numpy.dot instead
  sim_mat= dot(decoder_states, encoder_states.T)
<ipython-input-2-6de68eb61f38>:5: DeprecationWarning: scipy.dot is deprecated and will be removed in SciPy 2.0.0, use numpy.dot instead
  cont_vec = dot(att_mat, encoder_states)


In [3]:
print(f"a)\n {sim_mat}")
print(f"b)\n {att_mat}")
print(f"c)\n {cont_vec}")

a)
 [[  4.806   2.376   7.762   1.129]
 [ 12.164 -12.645  73.935   3.636]
 [ 27.79  -16.962  94.002   7.137]
 [ 18.702  -5.184  42.616   4.501]
 [ 64.38   49.86   56.21   14.45 ]]
b)
 [[4.91780633e-02 4.32948093e-03 9.45248312e-01 1.24414389e-03]
 [1.49003187e-27 2.50486173e-38 1.00000000e+00 2.94803216e-31]
 [1.75587568e-29 6.44090821e-49 1.00000000e+00 1.88369172e-38]
 [4.11416552e-11 1.74069934e-21 1.00000000e+00 2.79811669e-17]
 [9.99716568e-01 4.94220792e-07 2.82937800e-04 2.06801368e-22]]
c)
 [[ 9.69108631  0.35799187  0.59163688]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [ 1.20254471  3.39909302  5.59850122]]


Expected outputs:

a)  

    [[ 4.806 2.376 7.762 1.129] 
    [ 12.164 -12.645 73.935 3.636] 
    [ 27.79 -16.962 94.002 7.137] 
    [ 18.702 -5.184 42.616 4.501]
    [ 64.38 49.86 56.21 14.45 ]]
    

b) 

    [[4.91780633e-02 4.32948093e-03 9.45248312e-01 1.24414389e-03] 
    [1.49003187e-27 2.50486173e-38 1.00000000e+00 2.94803216e-31] 
    [1.75587568e-29 6.44090821e-49 1.00000000e+00 1.88369172e-38] 
    [4.11416552e-11 1.74069934e-21 1.00000000e+00 2.79811669e-17] 
    [9.99716568e-01 4.94220792e-07 2.82937800e-04 2.06801368e-22]]

c) 

    [[ 9.69108631 0.35799187 0.59163688] 
    [10.2 0.2 0.3 ] 
    [10.2 0.2 0.3 ]
    [10.2 0.2 0.3 ] 
    [ 1.20254471 3.39909302 5.59850122]]


# Transformer
## Using transformer-based T5 model to solve various NLP tasks in a sequence-to-sequence manner

Today we're going to learn a new library -- the HuggingFace **transformers** library (https://huggingface.co/docs/transformers/index) and use it to solve several non-obvious NLP-related problems using the **T5** model


HuggingFace transformers is one of the most popular libraries that provide us with a high-level API for using neural networks to solve tasks related to natural language processing, audio processing, computer vision, or even multimodal scenarios in which we have to utilize multiple modalities at once (e.g., answering questions about pictures, information extraction from invoices).

First, let's install the dependencies, the `transformers` library itself and the `sentencepiece` module, which helps us tokenize documents and transform tokens into one-hot encodings (we will discuss the idea of sentencepiece later in detail).

**Warning**: if you notice some weird exceptions like `cannot call from_pretrained on a None object` somewhere in your code, restart the environment using: Runtime -> restart. Then run the cells with code (without re-installing the libraries) one more time.

In [4]:
!pip install transformers   # install HuggingFace transformers library
!pip install sentencepiece  # install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The API provided by the `transformer` library is a high-level one. We can download a given model and generate an output using 4 lines of code!

Read the docs on the T5 model provided here: https://huggingface.co/docs/transformers/model_doc/t5

In the `inference` section, you can find a description showing how we can download a pretrained model, and use it to solve a given task. Simply use the code provided to translate some sentence from English into German!

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_ids = tokenizer("translate English to German: I like NLP.", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers --

Ich mag NLP.


## Various tasks

Experiment with some other inputs, e.g., those provided in Figure 1 presented in the paper introducing the T5 model or even a wider list of use cases from  Appendix D provided with the paper. You can find the paper here: https://arxiv.org/pdf/1910.10683.pdf

Note: there are some abbreviations used among the inputs provided, some of them are:
-  `stsb`: it stands for the semantic textual similarity benchmark. Given two sentences, we can calculate their semantic similarities, which can help us determine whether one sentence is a paraphrase of the other one.
-  `cola`: it stands for the Corpus of Linguistic Acceptability and helps us determine whether a given sentence is grammatical or ungrammatical.

If you look at Appendix D, there are more abbreviations, these are related to the names of tasks presented in the GLUE benchmark (available here: https://gluebenchmark.com/tasks) and the SUPERGLUE benchmark (available here: https://super.gluebenchmark.com/tasks). The idea of GLUE and SUPERGLUE is to collect a set of challenging tasks that may be used to evaluate the systems requiring natural language understanding. 

**Paste some 3 examples of tasks and the inputs you processed in the cell below** 

In [6]:
# TODO: paste 3 calls returning the outputs generated by T5 for some selected inputs
input_ids1 = tokenizer("summarize: state authorities dispatched emergency crews tuesday to survey the damage after an onslaught of severe weather in mississippi...", return_tensors="pt").input_ids
outputs1 = model.generate(input_ids1)
print(tokenizer.decode(outputs1[0], skip_special_tokens=True))

input_ids2 = tokenizer("stsb sentence1: The rhino grazed on the grass. sentence2: A rhino is grazing in a field.", return_tensors="pt").input_ids
outputs2 = model.generate(input_ids2)
print(tokenizer.decode(outputs2[0], skip_special_tokens=True))

input_ids3 = tokenizer("cola sentence: The course is jumping well.", return_tensors="pt").input_ids
outputs3 = model.generate(input_ids3)
print(tokenizer.decode(outputs3[0], skip_special_tokens=True))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


state authorities dispatched emergency crews to survey the damage. the damage was caused by
4.0
acceptable


## Various model types

There are several T5 models available, which differ in size (and quality). The bigger the model is, the better output it should generate. Experiment with some models from the following set: 
- t5-small
- t5-base
- t5-large
- t5-3b
- t5-11b

and check whether you can observe any difference in the quality of outputs.

Also, compare the size of the models, you can use the `model.num_parameters()` function to obtain the parameter number related to each model. For each model you are able to load, provide the size in the cell below (if you can't load a given model because it is too big, no worries, just type 'too big to load').

In [7]:
model_small = T5ForConditionalGeneration.from_pretrained("t5-small")
model_base = T5ForConditionalGeneration.from_pretrained("t5-base")
model_large = T5ForConditionalGeneration.from_pretrained("t5-large")
# model_3b = T5ForConditionalGeneration.from_pretrained("t5-3b") - too big to load
# model_11b = T5ForConditionalGeneration.from_pretrained("t5-11b") - too big to load

In [12]:
print(f"t5-small params number: {model_small.num_parameters()}")
print(f"t5-base params number: {model_base.num_parameters()}")
print(f"t5-large params number: {model_large.num_parameters()}")

t5-small params number: 60506624
t5-base params number: 222903552
t5-large params number: 737668096


## Language-specific T5s (OPTIONAL ASSIGNMENT -- you are not required to provide code here)

There are even some alternatives to the original T5 models. As the T5 model was trained on English, there are some models available that are specific to other languages, e.g., Polish (for example plT5 proposed by Allegro - https://huggingface.co/allegro/plt5-small). The Polish model was trained to solve a set of tasks collected in the KLEJ benchmark, which represents the Polish analogy to the GLUE benchmark: https://klejbenchmark.com.

You can find more details on plT5 in the research paper: https://arxiv.org/pdf/2205.08808.pdf. Table 2 presents some examples of prompts that can be used to solve some of the tasks listed in KLEJ.

You can search for an alternative to the original T5, for example, the one related to your language, and experiment with it (**this task is not mandatory**).

## Flan-T5

At the end of 2022, an evolution of T5 was proposed called Flan-T5. This model is also provided by the HuggingFace transformer library. Please visit this website: https://huggingface.co/docs/transformers/model_doc/flan-t5 to see how you can use this model (simply change the name of the model to download!). 

Flan-T5 is much more powerful than T5. You can take a look at Appendix D included in the paper describing Flan T5 to familiarize yourself with some input formats (prompts) and the generated values. The paper is here: https://arxiv.org/pdf/1910.10683.pdf. You should focus on `processed input` fields as they are the representations that the model consumes. Experiment with some selected tasks and see if you can obtain the same results! In the code below, paste some code loading the Flan-T5 model and using it to solve some selected tasks.

In [19]:
# TODO: Here paste some code using Flan-T5 model to solve some task
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

input_ids = tokenizer("What is ChatGPT?", return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_length=200, bos_token_id=0)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['ChatGPT ChatGPT is a telephonic service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service that is a service']


(not the best response xD)

In [23]:
input_ids = tokenizer("Who was the first president of the United States?", return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_length=200, bos_token_id=0)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['john w. bush']


It seems like flan-t5-small does not attain enough number of parameters ro provide correct answers.

## (OPTIONAL) Fine-tuning

You can even fine-tune the T5/Flan-T5 model to solve a task you want. You may load an existing T5/Flan-T5 model, which is already trained to solve some tasks, and use the power of transfer learning to learn it to solve some different tasks. This is much better than training a network from scratch and should require fewer training examples. 

The fine-tuning phase is quite complex. However, you can find the step-by-step description here: https://www.philschmid.de/fine-tune-flan-t5

You can try to fine-tune some selected model.